<a href="https://colab.research.google.com/github/iypc-team/NewNotebook/blob/master/New_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 11/5/2021-1
# https://www.tensorflow.org/tutorials/load_data/images
from __future__ import absolute_import
from IPython.display import clear_output
import glob, os, shutil, sys, uuid
from os.path import *
from time import sleep
# import TarfileFunctions
from TarfileFunctions import *
from BashColors import C
# from CV2_Utils import *
clear_output()
contentPath = os.getcwd()

os.chdir(contentPath)
cv2Path = join(contentPath, 'CV2Images')

if exists(cv2Path):
    # shutil.rmtree(cv2Path)
    sleep(0.05)

if not exists(cv2Path):
    os.mkdir(cv2Path)
    # tff.extractTarfiles('All_CV2_Images.tar.gz')
    sleep(0.05)

try:
    import tensorflow as tf
except Exception as err:
    print(err)
    %pip install tensorflow
    
import numpy as np

try: 
    from matplotlib import pyplot as plt
except Exception as err:
        %pip install matplotlib
        from matplotlib import pyplot as plt

os.chdir(contentPath)
clear_output()
import tensorflow as tf
print(f'tf version: {C.IBlue}{tf.__version__}{C.ColorOff}')
print(f'cwd: {C.IBlue}{os.getcwd()}{C.ColorOff}')

In [ ]:
tff.extractTarfiles('All_CV2_Images.tar.gz')

In [ ]:
os.chdir(cv2Path)
image_count = len(list(glob.glob('*/*.png', recursive=True)))
print(image_count, 'files')

In [ ]:
batch_size = 4
img_height = 224
img_width = 224

train_ds = tf.keras.utils.image_dataset_from_directory(
    cv2Path,
    color_mode='rgba',
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)
print()

val_ds = tf.keras.utils.image_dataset_from_directory(
    cv2Path,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)

In [ ]:
class_names = train_ds.class_names
print(class_names)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  for i in range(3):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

In [ ]:
for image_batch, labels_batch in train_ds:
    print(image_batch.shape)
    print(labels_batch.shape)
    break

In [ ]:
normalization_layer = tf.keras.layers.Rescaling(1./255)

In [ ]:
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
# Notice the pixel values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image))

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
num_classes = 2

model = tf.keras.Sequential([
  tf.keras.layers.Rescaling(1./255),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(num_classes)
])

In [ ]:
model.compile(
  optimizer='adam',
  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

In [ ]:
model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=3
)